In [2]:
import scipy.io

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = scipy.io.loadmat('/content/drive/My Drive/ASSIGNMENT 2/data5.mat')

In [5]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'x'])

In [6]:
data1 = data['x']

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
### in raw_data each column represent a different sample. There are 73 rows in this, last row represent output labels and remaining 72 rows
### are different features. There are 2148 columns meaning there are 2148 sample points.
raw_data = pd.DataFrame(data1.T)
raw_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147
0,28641.397310,11716.791410,27560.638120,12837.096380,9292.629213,27425.247190,6654.647725,14069.625980,9091.019171,17071.377250,43015.091530,10440.220390,13879.172850,4818.646313,12903.096020,4112.240203,7555.510451,29214.599930,12249.278210,11831.902190,7301.085206,23607.852250,15789.182590,84133.841700,80743.749120,13436.856150,20953.459120,16312.433570,12129.550170,7181.516404,6335.301490,19865.669340,25856.655960,10120.907250,18273.475630,28438.123270,62816.561940,20405.809570,38658.946090,24022.980620,...,22607.181150,7833.714015,20034.046520,16230.107350,10465.213170,24084.944750,13639.449110,25098.850300,60573.680230,160641.187900,28719.480080,4856.732171,29846.200520,53361.320780,18355.731040,56642.090280,125701.922100,16564.226740,2888.434539,27019.921450,44547.283800,34222.191840,18040.414300,69370.956050,18694.875520,58168.935940,7868.364849,25109.118460,43671.183330,30279.596090,20931.025920,21435.946640,22843.613580,49393.966050,3179.169903,2977.033370,5627.764176,114047.031800,3917.752407,13546.784640
1,34948.499340,38816.369540,39821.300750,52503.958480,13424.374940,59826.274080,254567.307200,76578.703300,6455.158853,13742.456560,165934.394300,28330.818160,83953.555030,7427.514425,16417.810700,17868.196380,6898.313510,68330.190520,27996.988920,42001.184930,12005.832130,20425.355500,18702.727580,166544.456300,224141.960400,28431.617240,44484.253900,299989.541300,86977.800430,31659.148370,6794.469297,30279.692370,37078.228440,127498.251800,61483.640290,50113.455350,18037.020600,22793.430290,95048.423610,38992.543180,...,12394.338310,4892.685840,418581.487400,42436.554320,60994.753440,20722.801130,11934.682770,21382.880380,60811.651740,155993.313300,35230.347310,29894.077890,22740.257740,35156.841920,47812.054250,36014.919600,13038.767270,12679.305640,3717.464795,40759.000350,16608.266980,17794.958520,23538.643210,15501.252650,26355.347030,38076.292580,31448.514910,13917.592330,26795.035020,48709.976840,5499.192034,28640.962360,7583.100198,34669.811730,53399.367460,105067.824800,136508.531800,3715.382499,14440.624870,22069.210510
2,3847.886139,8735.337066,38081.061940,26907.090880,4145.234774,10283.074950,248591.235500,42135.099340,1171.827810,2919.424989,44486.882510,8243.240174,48250.589670,1981.296327,5867.653441,12299.311750,757.926161,11886.322810,4425.364608,9854.063303,5554.198532,3930.238666,4709.664883,23255.587560,48539.909770,8856.188175,9826.699238,333734.018600,52009.914210,17894.071790,2242.462074,5911.538763,2500.655620,68027.583250,16384.487690,6749.469986,17955.155030,1834.435714,25999.127400,2899.524338,...,34684.440410,6816.613422,312181.136500,84645.661850,23656.256850,5213.988581,14927.064340,66674.289660,94313.500300,24161.544760,5014.486007,17199.739170,25714.575600,8759.132140,12934.480140,16515.405240,97644.588540,26278.247650,5423.472600,6959.166967,53455.650610,17126.899820,2718.560058,94540.272260,88211.824790,138737.457500,54896.918730,7348.311738,34275.429240,44126.666270,20307.301650,45200.743230,19608.382870,20134.764890,38053.059560,83318.929350,96434.958850,94195.580470,9090.925107,4456.275898
3,30885.682950,23089.079680,24186.561730,25929.476300,10320.885920,41065.502750,68476.301130,34795.919750,7484.160679,14664.835610,93329.074620,17355.749320,36829.431210,5638.972399,13195.561210,7918.522767,7038.810903,45804.368230,19022.343590,24482.502740,8269.511425,21048.013010,16079.125770,119617.564700,140286.912800,18706.351400,30254.699150,74722.952660,36556.387400,14949.260840,6002.153610,23594.019460,30846.892140,51778.216230,35788.757940,37590.571210,35930.191590,21178.048360,60338.328170,30785.297670,...,8821.284345,4650.109295,141399.245300,8177.792331,29829.597090,2109

In [9]:
### shuffling the data
shuffled_raw_data = raw_data.sample(frac = 1, random_state = 7, axis=1).reset_index(drop = True)
shuffled_raw_data.columns = list(range(raw_data.shape[1]))
shuffled_raw_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147
0,9099.953116,18545.804850,31221.136800,6045.378272,11386.552440,2468.029048,40585.112370,10144.683520,41038.504630,31822.221040,19925.187430,14069.625980,16683.410070,22520.057740,41315.970700,6113.170217,24931.096650,25994.024340,38249.789810,23866.593830,16107.264880,28075.543210,7047.109471,6070.891412,1179.567565,36860.232600,31750.789340,22102.103920,65635.577690,12096.310900,5508.673950,20152.433810,17100.588720,43693.261610,31008.997010,35598.746150,52945.225710,45030.438360,93306.034560,25419.605360,...,12533.848830,23178.576160,47975.671260,23477.648680,26171.719110,56006.673620,14437.678950,11996.806540,24213.345070,39760.503330,61682.074970,14833.860000,34429.205460,5000.548119,39572.957130,16617.980730,42572.772250,7566.007893,22534.137970,8707.592580,16632.387970,10325.524880,24823.144510,23283.335880,2370.842746,4357.666674,73693.359590,15838.636600,28946.536940,29974.754790,11955.089390,20224.746840,63761.223360,22276.943330,45450.381960,6796.516312,19811.680390,29283.205920,20215.280930,29142.509520
1,34419.448400,25605.492620,33857.898070,14044.801660,80965.113360,3295.677982,22404.359950,18163.662100,29168.533160,53395.171980,48069.733800,76578.703300,287500.841000,3754.443578,137846.880300,12865.763040,26805.664600,19158.029030,48430.817180,20559.113060,44003.807150,32394.208550,29863.088540,28529.163660,48955.236960,72655.327310,32936.267210,12550.863520,28070.771040,28241.353520,9015.150641,7801.354110,26930.673900,24504.110340,18229.446670,30666.065630,166704.868300,38689.096280,221806.030400,62974.090330,...,19440.199430,27568.521840,127115.418600,15900.476540,40546.016970,62573.914430,16642.457500,278424.182400,4866.320277,8310.973432,16321.713990,50625.023910,39857.276670,24913.796150,80483.075600,28442.626590,8486.314744,73459.153720,49413.721970,28964.725520,19414.312160,20329.308040,19004.234160,16942.109470,5373.603683,25050.588230,31628.537310,4539.945699,58797.840730,13121.141440,41633.102560,8394.099590,61662.241650,57471.372310,172426.543100,21456.627700,9095.749423,25639.635730,22626.776280,27918.902060
2,12591.255480,1668.275131,23264.267890,2527.740590,42101.232220,6675.408924,7666.949916,22084.831070,8404.194211,7228.038964,9851.947712,42135.099340,319101.053600,18858.004360,211014.082200,5630.895972,640.289972,68733.952240,40815.518570,20228.289610,43033.539760,45772.032120,51884.787460,9723.199147,37160.065200,15237.973810,3150.429313,23541.407660,19747.678450,26470.212700,5902.318618,15639.208950,56772.725660,4087.749660,44187.544390,343.908104,237975.910800,5445.026312,149647.966400,27588.450400,...,4459.485119,589.331937,187450.876800,52700.870180,13352.080280,10746.588220,2556.498637,258105.067400,24870.067380,48774.080840,42893.340560,11953.976030,19447.296320,12235.309300,10238.516260,7332.083110,16001.831310,50169.544900,5414.445016,7306.976362,5889.043570,31693.914300,30450.262770,35081.073090,705.997749,14893.790160,33054.896100,13985.761130,9516.354835,8479.313070,20935.185850,16671.576880,9501.053956,70742.957400,44271.891250,7195.320432,10838.061950,3921.840447,38847.549880,1169.222279
3,18620.637120,21654.124980,26745.169910,9413.666310,35655.300420,1212.129428,29570.949680,8628.528354,32991.572690,40799.576310,31530.953590,34795.919750,72326.385480,8417.519040,36873.269790,8084.788568,25707.753240,5427.835545,33137.027910,17133.753100,19313.089240,18782.629410,5493.877003,14870.318230,15696.453870,50934.129770,31560.107560,11442.457380,41921.022070,13579.036180,5789.098745,10069.580950,7798.507501,33141.578740,13567.869690,33048.431450,50391.826670,40508.209340,120133.630800,37306.872510,...,14875.343490,25174.592810,40745.438040,6500.680790,30020.984150,56648.1983

In [10]:
#### separating input data and output labels
X = shuffled_raw_data.iloc[:-1, :]
Y = shuffled_raw_data.iloc[-1:, :].reset_index(drop=True)
print("shape of X is: ", X.shape)
print("shape of Y is: ", Y.shape)

shape of X is:  (72, 2148)
shape of Y is:  (1, 2148)


In [11]:
def split(X, Y, val_split = 0.1, test_split = 0.2):
  '''
  splits the input and output in
  train, validation and test set
  '''
  train_size = int(X.shape[1] * (1 - (val_split + test_split)))
  val_size = int(X.shape[1] * val_split)
  test_size = X.shape[1] - train_size - val_size                     

  X_tr = X.iloc[:, :train_size]
  Y_tr = Y.iloc[:, :train_size]

  X_val = X.iloc[:, train_size:train_size + val_size]
  Y_val = Y.iloc[:, train_size:train_size + val_size]
  X_val.columns = list(range(val_size))                       ### to set column name from 0 to val_size
  Y_val.columns = list(range(val_size))

  X_ts = X.iloc[:, train_size + val_size:]
  Y_ts = Y.iloc[:, train_size + val_size:]
  X_ts.columns = list(range(test_size))                       ### to set column name from 0 to test_size
  Y_ts.columns = list(range(test_size))

  return X_tr, Y_tr, X_val, Y_val, X_ts, Y_ts

In [12]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = split(X, Y, val_split=0.1, test_split=0.2)
print("X_train shape is: ", X_train.shape)
print("Y_train shape is: ", Y_train.shape)
print("X_val shape is: ", X_val.shape)
print("Y_val shape is: ", Y_val.shape)
print("X_test shape is: ", X_test.shape)
print("Y_test shape is: ", Y_test.shape)

X_train shape is:  (72, 1503)
Y_train shape is:  (1, 1503)
X_val shape is:  (72, 214)
Y_val shape is:  (1, 214)
X_test shape is:  (72, 431)
Y_test shape is:  (1, 431)


In [113]:
min_X_train_feat = list(X_train.min(axis=1))
max_X_train_feat = list(X_train.max(axis=1))

In [114]:
X_train_norm = normalize(X_train, min_X_train_feat, max_X_train_feat)
X_val_norm = normalize(X_val, min_X_train_feat, max_X_train_feat)
X_test_norm = normalize(X_test, min_X_train_feat, max_X_train_feat)

In [231]:
### converting dataframe to numpy array to obtained trained parameters

#X_test_norm_arr = X_test_norm.to_numpy()
#X_val_norm_arr = X_val_norm.to_numpy()
X_train_norm_arr = X_train_norm.to_numpy()
Y_train_arr = Y_train.to_numpy()
#Y_test_arr = Y_test.to_numpy()
#Y_val_arr = Y_val.to_numpy()

In [160]:
def initialize_params(dims):
  '''
  dims: number of neurons in each layer
  returns:
  parameters: python dict containing W1, b1, w2, b2...., WL, bL
              W1--- weight matrix of shape(dims[l], dims[l-1])
              b1--- bias matrix of shape(dims[l], 1)

  '''
  np.random.seed(3)
  parameters = {}  ## empty dict
  L = len(dims)    ## number of layers in NN

  for l in range(1, L):
    parameters["W" + str(l)] = np.random.randn(dims[l], dims[l-1])
    parameters["b" + str(l)] = np.zeros((dims[l], 1))

  return parameters

In [133]:
### checking the initialize_params function
param = initialize_params([5,4,3])
print("W1 shape is: ", param["W1"].shape)
print("b1 shape is: ", param["b1"].shape)
print("W2 shape is: ", param["W2"].shape)
print("b2 shape is: ", param["b2"].shape)

W1 shape is:  (4, 5)
b1 shape is:  (4, 1)
W2 shape is:  (3, 4)
b2 shape is:  (3, 1)


In [134]:
def linear_forward(A, W, b):
  '''
  A: activations from previous layer
  W: weight matrix of shape(size of surrent layer, size of prev layer)
  b: bias matrix of shape(size_current_layer, 1)

  returns:
  Z: W*A + b --- the input of activation function
  cache: (A, W, b) --- for backpropagation
  '''
  Z = np.dot(W,A) + b
  cache = (A, W, b)

  return Z, cache

In [135]:
def sigmoid(Z):
  '''
  Z: array of any shape
  
  returns:
  A = sigmoid(Z)
  cache = Z ---- helpful in backprop
  '''
  A = 1/(1 + np.exp(-Z))
  cache = Z
  return A, cache

In [136]:
def relu(Z):
  '''
  Z: array of any shape

  returns:
  A = max(0, Z)
  cache = Z --- helpful in backprop
  '''
  A = np.maximum(0, Z)
  cache = Z
  return A, cache

In [137]:
def linear_activation_forward(A_prev, W, b, activation):
  '''
  A_prev: Activations from prev layer
  W: weight matrix
  b: bias matrix
  activation: either relu or sigmoid

  returns:
  A: activation values
  cache: (linear_cache, activation_cache)
  '''
  if activation == "sigmoid":
    Z, linear_cache = linear_forward(A_prev, W, b)     ### Z stores = A_prev * W + b, linear_cache stores = (A_prev, W, b)
    A, activation_cache = sigmoid(Z)                   ### A stores = sigmoid(Z),    activation_cache stores = (Z)

  elif activation == "relu":
    Z, linear_cache = linear_forward(A_prev, W, b)    ### Z stores = A_prev * W + b, linear_cache stores = (A_prev, W, b)
    A, activation_cache = relu(Z)                     ### ### A stores = sigmoid(Z),    activation_cache stores = (Z)

  cache = (linear_cache, activation_cache)            ### cache stores = ((A_prev, W, b), (Z))
  return A, cache                                     ### this function returns A, ((A_prev, W, b), (Z)) ---- helpful in backprop

In [138]:
def l_layer_forward(X, parameters):
  '''
  for l layered NN, this function will calculate relu activation for (l-1) layers and sigmoid activatio for last layer
  X: input
  parameters: initialized parameters

  returns:
  AL: last layer activation value
  caches: list of caches containing:
          linear_relu_cache (there will be l-1 of these caches ---- indexed from 0 to l-2)
          linear_sigmoid_cache (there will 1 cache  ------ indexed l-1)
  '''
  caches = []
  L = len(parameters)//2             ### number of layers in NN
  A = X                              ### Activation of input layer is X

  for l in range(1, L):             ### for l-1 relu layers
    A_prev = A
    A, cache = linear_activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], "relu")
    caches.append(cache)       ### caches stores ((A_prev, W, b), (Z)) for every layer

  AL, cache = linear_activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "sigmoid")  ### for sigmoid layer
  caches.append(cache)

  return AL, caches

In [139]:
def compute_cost(AL, Y):
  '''
  AL: activations of last layer of shape (1, number of examples)
  Y: actual output labels of shape (1, number of examples)

  returns:
  cost: cross_entropy loss
  '''
  m = Y.shape[1]
  epsilon = 1e-5
  cost_val = np.multiply(Y, np.log(AL+epsilon)) + np.multiply((1-Y), np.log(1-AL+epsilon))
  cost = -1/m * cost_val

  cost = np.squeeze(cost)         ### convert [[1.23]] in 1.23
  return cost

In [167]:
def linear_backward(dZ, cache):
  '''
  dZ: gradient of cost wrt Z of current layer (l)
  cache: (A_prev, W, b)
  
  returns:
  dW: gradient of cost wrt W (current_layer) ---- same shape as W
  db: gradient of bias wrt b (current layer) ---- same shape as b
  dA_prev: gradient of cost wrt to activation of previous layer ---- same shape as A_prev
  '''
  A_prev, W, b = cache
  m = A_prev.shape[1]

  dW = np.dot(dZ, A_prev.T)/m
  db = 1/m * np.sum(dZ,axis=1,keepdims = True)
  dA_prev = np.dot(W.T, dZ)
  
  return dA_prev, dW, db

In [168]:
def sigmoid_backward(dA, cache):
  '''
  dA: gradient of activation wrt cost ---- dA = dCost/dA
  cache: 'Z' that we stored earlier

  returns
  dZ: gradient of cost wrt to Z ----- dZ = dCost/dZ = dCost/dA * dA/dZ

  for sigmoid: dA/dZ = A*(1-A)
  '''
  Z = cache
  epsilon = 1e-3
  s = 1/(1+np.exp(-Z)+epsilon)
  dZ = dA * s * (1-s)

  return dZ

In [142]:
def relu_backward(dA, cache):
  '''
  dA: gradient of activation wrt cost ---- dA = dCost/dA
  cache: 'Z' that we stored earlier

  returns
  dZ: gradient of cost wrt to Z ----- dZ = dCost/dZ = dCost/dA * dA/dZ
  '''
  Z = cache
  dZ = np.array(dA, copy=True)
  dZ[Z<=0] = 0
  return dZ

In [169]:
def linear_activation_backward(dA, cache, activation):
  '''
  dA: gradient of activation wrt cost
  cache: (linear_cache, activation_cache)
  activation: either sigmoid or relu

  returns:
  dA_prev: gradient of cost wrt A_prev (layer l-1)
  dW: gradient of cost wrt W --- current layer
  db: gradient of cost wrt b --- current layer
  ''' 
  linear_cache, activation_cache = cache     #### linear_cache stores: (A_prev, W, b) and activation_cache stores: (Z)
  if activation == "sigmoid":
    dZ = sigmoid_backward(dA, activation_cache)

  elif activation == "relu":
    dZ = relu_backward(dA, activation_cache)

  dA_prev, dW, db = linear_backward(dZ, linear_cache)

  return dA_prev, dW, db

In [170]:
def l_layer_backward(AL, Y, caches):
  '''
  to initialize backprop, we need to calculate dAL ---- dAL = dCost/dAL

  AL: activation of last layer
  Y: actual output labels
  caches: list of caches containing:
          every linear_activation_cache with relu function --- indexed from 0 to l-2(because we have l-1 relu layers)
          linear_activation_cache with sigmoid function ---- indexed at L-1

  returns: 
  grads dict containing
  dA, dW, db
  '''
  grads = {}
  L = len(caches)
  m = Y.shape[1]
  Y = Y.reshape(AL.shape)
  epsilon = 1e-3
  dAL = -(np.divide(Y, AL+epsilon) - np.divide((1-Y), (1-AL+epsilon)))
  current_cache = caches[L-1]
  grads["dA"+str(L-1)], grads["dW"+str(L)], grads["db"+str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
  
  for l in reversed(range(L-1)):
    current_cache = caches[l]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)], current_cache, "relu")
    grads["dA"+str(l)] = dA_prev_temp
    grads["dW"+str(l+1)] = dW_temp
    grads["db"+str(l+1)] = db_temp
  
  return grads

In [171]:
def update_params(parameters, grads, learning_rate):
  '''
  parameters: dict containing parameters
  grads: dict containing gradients
  learning_rate: learning_rate of model

  returns:
  parameters: dict containing updated parameters
  '''
  L = len(parameters)//2
  for l in range(L):
    parameters["W"+str(l+1)] = parameters["W"+str(l+1)] - learning_rate * grads["dW"+str(l+1)]
    parameters["b"+str(l+1)] = parameters["b"+str(l+1)] - learning_rate * grads["db"+str(l+1)]

  return parameters

In [172]:
def model(X, Y, dims, learning_rate = 0.01, epochs = 1000):
  '''
  X and Y: input and output data
  dims: list containing number of neurons in each layer(including input layer)
  learning_rate: learning rate of gradient descent
  epochs: number of iterations 

  returns 
  parameters: final parameters of model
  '''
  np.random.seed(1)
  costs = []
  
  parameters = initialize_params(dims)
  for i in range(epochs):
    AL, caches = l_layer_forward(X, parameters)
    cost = compute_cost(AL, Y)
    costs.append(cost)
    gradients = l_layer_backward(AL, Y, caches)
    parameters = update_params(parameters, gradients, learning_rate)

  return parameters

In [28]:
def predict(X, Y, parameters):
  '''
  X: data set that we want to predict labels of 
  Y: actual labels of data set
  parameters: final parameters obtained after training

  returns:
  predicted labels
  '''
  m = X.shape[1]
  L = len(parameters)//2
  Y_pred = np.zeros((1, m))

  A_L, caches = l_layer_forward(X, parameters)
  #print(A_L)
  for i in range(m):
    if A_L[0,i] >= 0.5:
      Y_pred[0,i] = 1
    else:
      Y_pred[0,i] = 0

  return pd.DataFrame(Y_pred)

In [29]:
def confusion_matrix(Y, Y_pred):
  '''
  Y: actual labels
  Y_pred: predicted labels

  returns
  confusion_matrix
  '''
  df_compare = Y.copy()
  df_compare[1] = Y_pred
  df_compare.columns = ['Actual', 'Predicted']
  conf_mat = pd.crosstab(df_compare['Actual'], df_compare['Predicted'], rownames = ['Actual'], colnames = ['Predicted'])
  return conf_mat

In [30]:
def performance_measure(confusion_matrix):
  '''
  returns accuracy, specificity and sensitivity
  '''
  accuracy = (confusion_matrix[0][0] + confusion_matrix[1][1])/(np.sum(np.sum(confusion_matrix)))
  specificity = (confusion_matrix[0][0])/(np.sum(confusion_matrix[0]))
  sensitivity = (confusion_matrix[1][1])/(np.sum(confusion_matrix[1]))
  return accuracy, specificity, sensitivity

In [31]:
def normalize(X, min_features, max_features):
  n = X.shape[0]
  m = X.shape[1]
  X1 = pd.DataFrame(np.zeros((n,m)))
  for i in range(n):
    X1.iloc[i,:] = (X.iloc[i,:] - min_features[i])/(max_features[i] - min_features[i])
  return X1

In [224]:
parameters = model(X_train_norm_arr, Y_train_arr, dims = [X_train.shape[0], 36, 18, 1], learning_rate = 0.3, epochs = 2500)

In [225]:
Y_pred = predict(X_val_norm, Y_val, parameters)

In [226]:
conf_mat = confusion_matrix(Y_val.T, Y_pred.T)
conf_mat

Predicted,0.0,1.0
Actual,,
0.0,102,8
1.0,6,98


In [227]:
acc_val, speci_val, sensi_val = performance_measure(conf_mat)
print("Accuracy is: %f" %(acc_val*100)+' %')
print("Specificity is: %f" %(speci_val*100)+' %')
print("Sensitivity is: %f" %(sensi_val*100)+' %')

Accuracy is: 93.457944 %
Specificity is: 94.444444 %
Sensitivity is: 92.452830 %


In [228]:
Y_pred_test = predict(X_test_norm, Y_test, parameters)

In [229]:
conf_mat_test = confusion_matrix(Y_test.T, Y_pred_test.T)
conf_mat_test

Predicted,0.0,1.0
Actual,,
0.0,196,12
1.0,20,203


In [230]:
acc_test, speci_test, sensi_test = performance_measure(conf_mat_test)
print("Accuracy is: %f" %(acc_test*100)+' %')
print("Specificity is: %f" %(speci_test*100)+' %')
print("Sensitivity is: %f" %(sensi_test*100)+' %')

Accuracy is: 92.575406 %
Specificity is: 90.740741 %
Sensitivity is: 94.418605 %
